<a href="https://colab.research.google.com/github/philipayazi/Disaster_Tweets/blob/master/Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [2]:
import pickle

# Getting back the objects:
with open('objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    test_tweet_matrix, train_tweet_matrix, y_train_split, y_test_split, nlp = pickle.load(f)

## DL Classifier from W2V Features

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [4]:
X_train =train_tweet_matrix
X_test =test_tweet_matrix
y_train =y_train_split
y_test =y_test_split

print('Train X and y', X_train.shape, y_train.shape)
print('Test X and y', X_test.shape, y_test.shape)

Train X and y (5329, 300) (5329,)
Test X and y (2284, 300) (2284,)


In [5]:
#---------------- No scaling gives better results
X_train_scaled =X_train
X_test_scaled =X_test

In [6]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [7]:
y_train_categorical.shape

(5329, 2)

### Create DL model

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [35]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=300))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [36]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 40,402
Trainable params: 40,402
Non-trainable params: 0
_________________________________________________________________


In [38]:
#fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=10,  #60,
    shuffle=True,
    verbose=2,
    validation_split=0.15
)

Train on 4529 samples, validate on 800 samples
Epoch 1/10
4529/4529 - 1s - loss: 0.5006 - accuracy: 0.7673 - val_loss: 0.4831 - val_accuracy: 0.7812
Epoch 2/10
4529/4529 - 0s - loss: 0.4156 - accuracy: 0.8161 - val_loss: 0.4507 - val_accuracy: 0.7887
Epoch 3/10
4529/4529 - 0s - loss: 0.3886 - accuracy: 0.8287 - val_loss: 0.4572 - val_accuracy: 0.7950
Epoch 4/10
4529/4529 - 0s - loss: 0.3510 - accuracy: 0.8547 - val_loss: 0.4820 - val_accuracy: 0.8012
Epoch 5/10
4529/4529 - 1s - loss: 0.3156 - accuracy: 0.8680 - val_loss: 0.4686 - val_accuracy: 0.7850
Epoch 6/10
4529/4529 - 0s - loss: 0.2846 - accuracy: 0.8836 - val_loss: 0.5012 - val_accuracy: 0.7950
Epoch 7/10
4529/4529 - 0s - loss: 0.2523 - accuracy: 0.8975 - val_loss: 0.5181 - val_accuracy: 0.8025
Epoch 8/10
4529/4529 - 0s - loss: 0.2219 - accuracy: 0.9121 - val_loss: 0.5999 - val_accuracy: 0.7862
Epoch 9/10
4529/4529 - 0s - loss: 0.1828 - accuracy: 0.9311 - val_loss: 0.6561 - val_accuracy: 0.8025
Epoch 10/10
4529/4529 - 0s - loss: 

In [39]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=3)
print(
    f"RESULTS - Loss: {model_loss}, Accuracy: {model_accuracy}")

RESULTS - Loss: 0.6603593230247498, Accuracy: 0.793345034122467


In [40]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [41]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [0 0 0 0 0]
Actual Labels: [1, 0, 1, 0, 0]


In [42]:
model.evaluate(X_test_scaled, y_test_categorical, verbose=1)

2284/2284 [==============================] - 0s 43us/sample - loss: 0.6604 - accuracy: 0.7933


[0.6603593230247498, 0.79334503]

In [52]:
## ---------- Pred Rate, Prec Rate, F1 score

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix

y_pred1 = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred1, axis=1)

# Print f1, precision, and recall scores
y_test1 = y_test

print('ACC: ', accuracy_score(y_test1, y_pred))

print('RE: ', recall_score(y_test1, y_pred , average="macro"))
print('PR: ', precision_score(y_test1, y_pred , average="macro"))
print('F1: ', f1_score(y_test1, y_pred , average="macro"))

ACC:  0.7933450087565674
RE:  0.791773092426256
PR:  0.7883532253657568
F1:  0.7896677210127028


## Hyperas ... for hyperparams optimization

In [53]:
def data():
    with open('objs.pkl', 'rb') as f:  
        test_tweet_matrix, train_tweet_matrix, y_train_split, y_test_split, nlp = pickle.load(f)

    X_train =train_tweet_matrix
    X_test =test_tweet_matrix
        
    # Step 1: Label-encode data set
    label_encoder = LabelEncoder()
    label_encoder.fit(y_train_split)
    encoded_y_train = label_encoder.transform(y_train_split)
    encoded_y_test = label_encoder.transform(y_test_split)

    # Step 2: Convert encoded labels to one-hot-encoding
    y_train_categorical = to_categorical(encoded_y_train)
    y_test_categorical = to_categorical(encoded_y_test)

    y_train =y_train_categorical
    y_test =y_test_categorical 
    
    X_scaler = StandardScaler().fit(X_train)
    
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    y_train_scaled = y_train
    y_test_scaled = y_test

    x_train = X_train_scaled
    x_test = X_test_scaled
    y_train = y_train_scaled
    y_test = y_test_scaled
   
    return x_train, y_train, x_test, y_test

In [56]:
from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras import backend as K
from keras.utils import print_summary
          
              
def create_model(x_train, y_train, x_test, y_test):
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    
    model= Sequential() 
    model.add(Dense( {{choice([50,100,500])}} , activation='relu', input_dim=300))
    model.add(Dense( {{choice([50,100,500])}} , activation='relu'))
    model.add(Dense(units=2, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])   
    print_summary(model, line_length=None, positions=None, print_fn=None)
    result= model.fit(x_train, y_train,
                      batch_size={{choice([64,128])}},
                      epochs={{choice([10,50,100])}},
                      verbose=2,
                      validation_split =0.15)
    
    score = model.evaluate(x_test, y_test, verbose=0)
    accuracy = score[1]
  #  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}
    return {'loss': accuracy, 'status': STATUS_OK, 'model': model}

best_run, best_model= optim.minimize(model=create_model,
                                     data=data,
                                     algo=tpe.suggest,
                                     max_evals=5,
                                     trials=Trials(),
                                     notebook_name='Disaster_Tweets_hyperasW2VDL')


>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.metrics import classification_report, confusion_matrix
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    import pickle
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import LabelEncoder, MinMaxScaler
except:
    pass

try:
    from tensorflow.keras.utils import to_categorical
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

try:
    from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass


 - 0s - loss: 0.1781 - accuracy: 0.9410 - val_loss: 0.5089 - val_accuracy: 0.7975                                                                

Epoch 6/100                                                                                                                                      
 - 0s - loss: 0.1384 - accuracy: 0.9580 - val_loss: 0.5522 - val_accuracy: 0.7925                                                                

Epoch 7/100                                                                                                                                      
 - 0s - loss: 0.1154 - accuracy: 0.9658 - val_loss: 0.5833 - val_accuracy: 0.8000                                                                

Epoch 8/100                                                                                                                                      
 - 0s - loss: 0.1022 - accuracy: 0.9750 - val_loss: 0.6477 - val_accuracy: 0.7850                                        

Epoch 33/100                                                                                                                                     
 - 0s - loss: 0.0399 - accuracy: 0.9841 - val_loss: 0.9193 - val_accuracy: 0.8037                                                                

Epoch 34/100                                                                                                                                     
 - 0s - loss: 0.0383 - accuracy: 0.9826 - val_loss: 0.9337 - val_accuracy: 0.8037                                                                

Epoch 35/100                                                                                                                                     
 - 0s - loss: 0.0406 - accuracy: 0.9828 - val_loss: 0.9322 - val_accuracy: 0.7975                                                                

Epoch 36/100                                                                                                             

 - 0s - loss: 0.0329 - accuracy: 0.9845 - val_loss: 1.1037 - val_accuracy: 0.7987                                                                

Epoch 61/100                                                                                                                                     
 - 0s - loss: 0.0290 - accuracy: 0.9839 - val_loss: 1.0741 - val_accuracy: 0.8012                                                                

Epoch 62/100                                                                                                                                     
 - 0s - loss: 0.0288 - accuracy: 0.9830 - val_loss: 1.0973 - val_accuracy: 0.8000                                                                

Epoch 63/100                                                                                                                                     
 - 0s - loss: 0.0327 - accuracy: 0.9837 - val_loss: 1.1781 - val_accuracy: 0.7950                                        

Epoch 88/100                                                                                                                                     
 - 0s - loss: 0.0271 - accuracy: 0.9848 - val_loss: 1.2603 - val_accuracy: 0.7950                                                                

Epoch 89/100                                                                                                                                     
 - 0s - loss: 0.0256 - accuracy: 0.9852 - val_loss: 1.2713 - val_accuracy: 0.8000                                                                

Epoch 90/100                                                                                                                                     
 - 0s - loss: 0.0265 - accuracy: 0.9848 - val_loss: 1.2622 - val_accuracy: 0.7950                                                                

Epoch 91/100                                                                                                             

...

Epoch 14/50                                                                                                                                      
 - 0s - loss: 0.0798 - accuracy: 0.9755 - val_loss: 0.8553 - val_accuracy: 0.7812                                                                

Epoch 15/50                                                                                                                                      
 - 0s - loss: 0.0690 - accuracy: 0.9788 - val_loss: 0.9133 - val_accuracy: 0.7825                                                                

Epoch 16/50                                                                                                                                      
 - 0s - loss: 0.0697 - accuracy: 0.9762 - val_loss: 0.8998 - val_accuracy: 0.7713                                                                

Epoch 17/50                                                                                                              

 - 0s - loss: 0.0368 - accuracy: 0.9817 - val_loss: 1.1452 - val_accuracy: 0.7825                                                                

Epoch 42/50                                                                                                                                      
 - 0s - loss: 0.0364 - accuracy: 0.9828 - val_loss: 1.2338 - val_accuracy: 0.7862                                                                

Epoch 43/50                                                                                                                                      
 - 0s - loss: 0.0365 - accuracy: 0.9819 - val_loss: 1.2204 - val_accuracy: 0.7825                                                                

Epoch 44/50                                                                                                                                      
 - 0s - loss: 0.0362 - accuracy: 0.9821 - val_loss: 1.2350 - val_accuracy: 0.7850                                        

 - 0s - loss: 0.1538 - accuracy: 0.9503 - val_loss: 0.5721 - val_accuracy: 0.7912                                                                

Epoch 10/50                                                                                                                                      
 - 0s - loss: 0.1306 - accuracy: 0.9609 - val_loss: 0.6012 - val_accuracy: 0.8012                                                                

Epoch 11/50                                                                                                                                      
 - 0s - loss: 0.1121 - accuracy: 0.9693 - val_loss: 0.6364 - val_accuracy: 0.7875                                                                

Epoch 12/50                                                                                                                                      
 - 0s - loss: 0.0992 - accuracy: 0.9748 - val_loss: 0.6414 - val_accuracy: 0.7950                                        

Epoch 37/50                                                                                                                                      
 - 0s - loss: 0.0460 - accuracy: 0.9832 - val_loss: 0.9993 - val_accuracy: 0.7862                                                                

Epoch 38/50                                                                                                                                      
 - 0s - loss: 0.0422 - accuracy: 0.9841 - val_loss: 0.9406 - val_accuracy: 0.7875                                                                

Epoch 39/50                                                                                                                                      
 - 0s - loss: 0.0442 - accuracy: 0.9803 - val_loss: 0.9443 - val_accuracy: 0.7887                                                                

Epoch 40/50                                                                                                              

Epoch 5/10                                                                                                                                       
 - 0s - loss: 0.1508 - accuracy: 0.9481 - val_loss: 0.6280 - val_accuracy: 0.7775                                                                

Epoch 6/10                                                                                                                                       
 - 0s - loss: 0.1277 - accuracy: 0.9600 - val_loss: 0.6511 - val_accuracy: 0.7850                                                                

Epoch 7/10                                                                                                                                       
 - 0s - loss: 0.1078 - accuracy: 0.9664 - val_loss: 0.7267 - val_accuracy: 0.7550                                                                

Epoch 8/10                                                                                                               